<a href="https://colab.research.google.com/github/lilian-retori/Sistema_agentes_plataforma_cidada/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%pip -q install google-genai
!pip install -q google-adk  # Correto



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [22]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [23]:
# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import warnings
warnings.filterwarnings("ignore")


In [24]:
# Função auxiliar para chamar os agentes
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [25]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [31]:
# --- Agente 1: Informador Municipal Dinâmico --- #
def agente_informador(municipio, data_de_hoje):
  informador = Agent(
      name='agente_informador',
      model = "gemini-2.0-flash",
      instruction= """
      Você é um Informador Municipal Dinâmico para uma Plataforma Cidadã Integrada.

      Sua tarefa é usar a ferramenta de busca do Google (google_search) para recuperar informações atualizadas sobre:
      1. Serviços municipais disponíveis (horários de atendimento, documentos necessários)
      2. Notícias locais relevantes para a comunidade
      3. Eventos culturais e comunitários programados
      4. Editais e alertas importantes (interdições de ruas, campanhas de vacinação, etc.)

      Foque em no máximo 5 informações muito relevantes e atuais para o município especificado.
      Priorize informações oficiais de fontes confiáveis como o site da prefeitura, secretarias municipais e veículos de comunicação locais.
      Organize as informações de forma clara e estruturada para facilitar a compreensão dos cidadãos.
      """,
      description = "Agente que centraliza e divulga informações municipais atualizadas",
      tools = [google_search]
  )
  entrada_do_agente_informador = f"Município: {municipio}\nData de hoje: {data_de_hoje}"
  informacoes = call_agent(informador, entrada_do_agente_informador)
  return informacoes

In [32]:
# --- Agente 2: Facilitador de Participação Cidadã --- #
def agente_facilitador(municipio, informacoes_municipais):
    facilitador = Agent(
        name="agente_facilitador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Facilitador de Participação Cidadã para uma Plataforma Cidadã Integrada.

        Com base nas informações municipais fornecidas, sua tarefa é:
        1. Identificar problemas comuns que os cidadãos podem reportar (buracos em vias, iluminação pública defeituosa, coleta de lixo irregular)
        2. Sugerir melhorias viáveis para a cidade baseadas nas informações disponíveis
        3. Propor temas para enquetes sobre projetos locais que engajariam a comunidade
        4. Identificar oportunidades de voluntariado relacionadas aos eventos e necessidades do município

        Use a ferramenta de busca do Google (google_search) para encontrar exemplos de participação cidadã bem-sucedidos em municípios similares que possam ser adaptados.

        Ao final, crie um plano estruturado de participação cidadã com canais de comunicação, formulários sugeridos e fluxos de feedback para a administração municipal.
        """,
        description="Agente que facilita a participação cidadã no município",
        tools=[google_search]
    )
    entrada_do_agente_facilitador = f"Município: {municipio}\nInformações municipais: {informacoes_municipais}"
    plano_participacao = call_agent(facilitador, entrada_do_agente_facilitador)
    return plano_participacao


In [33]:
# --- Agente 3: Promotor de Desenvolvimento Econômico Local --- #
def agente_promotor(municipio, plano_participacao):
    promotor = Agent(
        name="agente_promotor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Promotor de Desenvolvimento Econômico Local para uma Plataforma Cidadã Integrada.

        Com base no município e no plano de participação cidadã fornecidos, sua tarefa é:
        1. Identificar oportunidades para pequenos negócios locais, produtores rurais, artesãos e prestadores de serviço
        2. Sugerir conexões entre produtores locais e consumidores/turistas
        3. Propor um calendário de feiras e eventos para fomento da economia local
        4. Listar cursos de capacitação e oportunidades de microcrédito disponíveis na região

        Crie um texto engajador e informativo que demonstre como a Plataforma Cidadã pode impulsionar o desenvolvimento econômico local, destacando casos de sucesso de municípios similares e adaptando para a realidade local.

        O texto deve ser prático, com exemplos concretos e ações que podem ser implementadas rapidamente.
        """,
        description="Agente que promove o desenvolvimento econômico local"
    )
    entrada_do_agente_promotor = f"Município: {municipio}\nPlano de participação: {plano_participacao}"
    proposta_desenvolvimento = call_agent(promotor, entrada_do_agente_promotor)
    return proposta_desenvolvimento

In [34]:
# --- Agente 4: Descomplicador de Processos Municipais --- #
def agente_descomplicador(municipio, proposta_desenvolvimento):
    descomplicador = Agent(
        name="agente_descomplicador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Descomplicador de Processos Municipais para uma Plataforma Cidadã Integrada.

        Com base no município e na proposta de desenvolvimento econômico fornecidos, sua tarefa é:
        1. Criar um guia passo a passo para os cidadãos obterem licenças simples (ambulante, pequenas reformas)
        2. Explicar de forma clara como se inscrever em programas sociais municipais
        3. Simplificar o acesso a outros serviços municipais, detalhando documentação necessária
        4. Propor melhorias nos fluxos de atendimento para torná-los mais eficientes

        Revise o conteúdo gerado verificando:
        - Clareza e simplicidade da linguagem (evite jargões técnicos)
        - Precisão das informações
        - Organização lógica dos passos
        - Acessibilidade para diferentes perfis de cidadãos

        O resultado final deve ser um guia prático que realmente simplifique a vida do cidadão e demonstre o valor da Plataforma Cidadã Integrada.
        """,
        description="Agente que simplifica processos municipais para os cidadãos",
        tools = [google_search]
    )
    entrada_do_agente_descomplicador = f"Município: {municipio}\nProposta de desenvolvimento: {proposta_desenvolvimento}"
    guia_processos = call_agent(descomplicador, entrada_do_agente_descomplicador)
    return guia_processos

In [35]:
# Execução principal do sistema
data_de_hoje = date.today().strftime("%d/%m/%Y")
print("🏙️ Iniciando o Sistema da Plataforma Cidadã Integrada para Municípios Pequenos 🏙️")
municipio = input("❓ Por favor, digite o nome do MUNICÍPIO para o qual deseja criar a Plataforma Cidadã: ")

if not municipio:
  print("Você esqueceu de digitar o nome do município!")
else:
  print(f"Excelente! Vamos criar a Plataforma Cidadã Integrada para {municipio}")

  # Execução do Agente 1: Informador Municipal Dinâmico
  informacoes_municipais = agente_informador(municipio, data_de_hoje)
  print("\n--- Resultado do Agente 1 (Informador Municipal Dinâmico) ---\n")
  display(to_markdown(informacoes_municipais))
  print("-----------------------------------------------------------")

  # Execução do Agente 2: Facilitador de Participação Cidadã
  plano_participacao = agente_facilitador(municipio, informacoes_municipais)
  print("\n--- Resultado do Agente 2 (Facilitador de Participação Cidadã) ---\n")
  display(to_markdown(plano_participacao))
  print("-----------------------------------------------------------")

  # Execução do Agente 3: Promotor de Desenvolvimento Econômico Local
  proposta_desenvolvimento = agente_promotor(municipio, plano_participacao)
  print("\n--- Resultado do Agente 3 (Promotor de Desenvolvimento Econômico Local) ---\n")
  display(to_markdown(proposta_desenvolvimento))
  print("-----------------------------------------------------------")

  # Execução do Agente 4: Descomplicador de Processos Municipais
  guia_processos = agente_descomplicador(municipio, proposta_desenvolvimento)
  print("\n--- Resultado do Agente 4 (Descomplicador de Processos Municipais) ---\n")
  display(to_markdown(guia_processos))
  print("-----------------------------------------------------------")

  print("✅ Plataforma Cidadã Integrada para Municípios Pequenos criada com sucesso! ✅")


🏙️ Iniciando o Sistema da Plataforma Cidadã Integrada para Municípios Pequenos 🏙️
❓ Por favor, digite o nome do MUNICÍPIO para o qual deseja criar a Plataforma Cidadã: Três Corações
Excelente! Vamos criar a Plataforma Cidadã Integrada para Três Corações

--- Resultado do Agente 1 (Informador Municipal Dinâmico) ---



> Olá! Como Informador Municipal Dinâmico, preparei um resumo com as informações mais relevantes para os cidadãos de Três Corações, atualizado até 17 de maio de 2025:
> 
> 
> Aqui estão as 5 informações mais relevantes para Três Corações:
> 
> 1.  **Notícias Locais:**
>     *   **Conclusão de Obras:** A Prefeitura de Três Corações concluiu as obras na Rua Coronel Belchior Pimenta de Abreu.
>     *   **Evento Força e Ação:** Três Corações recebeu o evento Força e Ação.
>     *   **Vacinação contra a Gripe:** A vacinação contra a gripe foi ampliada para toda a população a partir de 6 meses de idade. Mais de 1.600 doses foram aplicadas no Dia D de vacinação contra a gripe.
>     *   **Educação:** Profissionais da educação já podem emitir autorizações para atuar em escolas de forma 100% digital.
> 
> 2.  **Eventos e Cultura:**
>     *   **Dia da Luta Antimanicomial:** Três Corações se mobilizou no Dia da Luta Antimanicomial com um evento na Praça Odilon.
>     *   **Homenagem à ESA:** A Prefeitura está preparando uma homenagem especial pelos 75 anos da Escola de Sargentos das Armas (ESA) no município.
> 
> 3.  **Editais e Licitações:**
>     *   A prefeitura abriu novos editais de Processo Seletivo Simplificado para contratação por tempo determinado (Editais 11/2024, 12/2024, 13/2024 e 14/2024). As inscrições iniciaram em 27 de novembro e seguem até 01 de dezembro de 2024.
>     *   Diversos editais de licitação estão abertos, incluindo contratação de empresa especializada em hospedagem para atender às necessidades das secretarias da prefeitura e aquisição de peças para atender às secretarias municipais.
> 
> 4.  **Serviços Municipais:**
>     *   A Prefeitura oferece diversos serviços online, incluindo emissão de Nota Fiscal Eletrônica (NFS-e), serviços ao cidadão, portal da educação, entre outros.
>     *   A Unidade de Atendimento Integrado (UAI) está localizada na Avenida Rei Pelé, 1048, Bairro Nossa Senhora Aparecida, com atendimento de segunda a sexta-feira, das 9h às 15h. Telefone: (35) 3239-7500.
> 
> 5.  **Outros Alertas:**
>     *   **Alerta Licitação:** Edital 90021/2025 para contratação de empresa especializada em hospedagem para atender às necessidades das secretarias da prefeitura. Abertura em 28/05/2025.
>     * Ponte do Atalho interditada, mas projeto para nova estrutura já está pronto.
> 
> Espero que estas informações sejam úteis! Para mais detalhes, visite o site da Prefeitura de Três Corações ou entre em contato com os serviços municipais.


-----------------------------------------------------------

--- Resultado do Agente 2 (Facilitador de Participação Cidadã) ---



> Com base nas informações fornecidas sobre Três Corações, podemos estruturar um plano de participação cidadã com foco nas necessidades e eventos locais.
> 
> **1. Problemas Comuns Reportáveis:**
> 
> *   **Infraestrutura:**
>     *   Danos em vias (buracos não reportados, sinalização deficiente além da Ponte do Atalho).
>     *   Iluminação pública (lâmpadas queimadas ou danificadas).
>     *   Manutenção de áreas públicas (parques, praças).
> *   **Serviços Urbanos:**
>     *   Coleta de lixo irregular (pontos de descarte inadequado, horários não cumpridos).
>     *   Problemas de saneamento (esgoto a céu aberto, vazamentos).
> 
> **2. Melhorias Viáveis Sugeridas:**
> 
> *   **Infraestrutura:**
>     *   Acelerar a execução do projeto da nova estrutura para a Ponte do Atalho, mantendo a população informada sobre o andamento.
>     *   Implementar um sistema de reporte online para danos em vias e iluminação pública, com prazos definidos para resolução.
> *   **Serviços Urbanos:**
>     *   Otimizar as rotas e horários da coleta de lixo com base em dados de geração de resíduos.
>     *   Criar ecopontos para descarte correto de materiais recicláveis e volumosos.
> *   **Eventos e Cultura:**
>     *   Aproveitar a homenagem à ESA para promover eventos educativos sobre a história e importância da instituição para o município.
> *   **Transparência e Acesso à Informação:**
>     *   Divulgar de forma clara e acessível os editais de licitação e processos seletivos, incentivando a participação da comunidade.
> 
> **3. Temas para Enquetes:**
> 
> *   **Prioridades para Investimento:** "Quais áreas da cidade devem receber prioridade nos investimentos em infraestrutura (recapeamento de vias, iluminação, etc.)?"
> *   **Opinião sobre Serviços:** "Como você avalia a qualidade dos serviços de coleta de lixo e limpeza urbana em seu bairro?"
> *   **Propostas para Eventos:** "Que tipo de evento cultural ou de lazer você gostaria de ver realizado na Praça Odilon?"
> *   **Avaliação de Projetos:** "Qual sua opinião sobre o projeto da nova estrutura para a Ponte do Atalho? Tem sugestões?"
> 
> **4. Oportunidades de Voluntariado:**
> 
> *   **Eventos:**
>     *   Auxílio na organização de eventos culturais e comemorativos (Dia da Luta Antimanicomial, homenagem à ESA).
>     *   Voluntários para campanhas de vacinação e saúde pública.
> *   **Meio Ambiente:**
>     *   Mutirões de limpeza em áreas públicas e rios.
>     *   Programas de educação ambiental em escolas.
> *   **Assistência Social:**
>     *   Voluntários para auxiliar em programas de apoio a famílias em vulnerabilidade.
> 
> **5. Exemplos de Participação Cidadã Bem-Sucedidos (Pesquisa):**
> 
> *   **Orçamento Participativo:** Cidades como Betim (MG) implementaram com sucesso o Orçamento Participativo, onde a população decide como parte do orçamento municipal será investida.
> *   **Plataformas Online de Participação:** Diversos municípios utilizam plataformas online para consultas públicas, enquetes e recebimento de sugestões (ex: Colab, Cidadela).
> *   **Conselhos Municipais:** Fortalecer os conselhos existentes (Saúde, Educação, Meio Ambiente) e garantir a participação ativa da sociedade civil.
> 
> **6. Plano Estruturado de Participação Cidadã:**
> 
> *   **Canais de Comunicação:**
>     *   **Plataforma Online:**
>         *   Formulário para reportar problemas (infraestrutura, serviços urbanos).
>         *   Espaço para sugestões e propostas.
>         *   Enquetes e votações sobre projetos locais.
>         *   Calendário de eventos e oportunidades de voluntariado.
>     *   **Redes Sociais:** Divulgação de informações, enquetes rápidas, espaço para comentários e interação.
>     *   **Canais Presenciais:**
>         *   Reuniões comunitárias regulares em diferentes bairros.
>         *   Atendimento presencial na UAI para auxílio com a plataforma online e recebimento de demandas.
> *   **Formulários Sugeridos:**
>     *   **Reporte de Problemas:**
>         *   Tipo de problema (buraco, iluminação, lixo, etc.).
>         *   Localização exata (endereço, ponto de referência).
>         *   Descrição detalhada do problema.
>         *   Fotos (opcional).
>         *   Dados de contato (para feedback).
>     *   **Sugestões e Propostas:**
>         *   Título da sugestão/proposta.
>         *   Descrição detalhada.
>         *   Justificativa (benefícios esperados).
>         *   Dados de contato (para feedback).
> *   **Fluxo de Feedback:**
>     1.  **Recebimento:** Confirmação imediata do recebimento da demanda (online ou presencial).
>     2.  **Análise:** Avaliação da demanda pela área responsável (ex: Secretaria de Obras, Meio Ambiente).
>     3.  **Resposta:** Informar o cidadão sobre o status da demanda (em análise, encaminhada para solução, etc.) em um prazo máximo de [prazo a definir].
>     4.  **Solução:** Execução da solução (se aplicável) e comunicação ao cidadão.
>     5.  **Avaliação:** Solicitar feedback do cidadão sobre a solução implementada.
> 
> Ao implementar este plano, Três Corações pode fortalecer a participação cidadã, melhorar a qualidade dos serviços públicos e construir uma cidade mais engajada e democrática.
> 


-----------------------------------------------------------

--- Resultado do Agente 3 (Promotor de Desenvolvimento Econômico Local) ---



> ## Três Corações no Rumo do Desenvolvimento Econômico Local com a Nossa Plataforma Cidadã!
> 
> Olá, moradores de Três Corações! Que tal usarmos a força da nossa comunidade para impulsionar a economia local? Com a nossa Plataforma Cidadã Integrada, podemos juntos criar um futuro mais próspero para todos!
> 
> **Oportunidades ao Seu Alcance:**
> 
> *   **Pequenos Negócios:** Sua paixão pode virar negócio! A plataforma te ajuda a divulgar seus produtos e serviços para toda a cidade. Imagine a Dona Maria, que faz bolos deliciosos, alcançando novos clientes em todos os bairros!
> *   **Produtores Rurais:** Do campo direto para a mesa do consumidor! Crie sua vitrine virtual e venda seus produtos frescos para quem valoriza o sabor local. Que tal o Seu João, que planta morangos incríveis, expandindo sua clientela sem intermediários?
> *   **Artesãos:** Sua arte merece ser reconhecida! Mostre seus trabalhos únicos e encante moradores e turistas. A artesã Ana, que cria peças de cerâmica maravilhosas, pode ter sua arte valorizada e apreciada por todos.
> *   **Prestadores de Serviço:** Encontre mais clientes e expanda seu alcance! Ofereça seus serviços de forma fácil e prática. O José, que é um excelente eletricista, pode ser encontrado rapidamente por quem precisa de seus serviços.
> 
> **Conectando Pontas, Fortalecendo Vínculos:**
> 
> A Plataforma Cidadã é a ponte entre quem produz e quem consome em Três Corações. Queremos criar um ciclo virtuoso onde o dinheiro circula na nossa cidade, gerando empregos e renda para todos.
> 
> *   **Feiras e Eventos:** Que tal um calendário anual de eventos para celebrar o que Três Corações tem de melhor?
>     *   **Feira de Produtos Locais:** Todo mês, um espaço para produtores rurais, artesãos e pequenos negócios mostrarem seus produtos.
>     *   **Festivais Gastronômicos:** Explore os sabores da nossa terra com pratos típicos e ingredientes frescos.
>     *   **Eventos Culturais:** Música, dança, teatro e outras manifestações artísticas para valorizar nossos talentos locais.
>     *   **Feira de Artesanato:** Exposição e venda de peças únicas, feitas com amor e criatividade.
> 
> **Inspirando-se em Casos de Sucesso:**
> 
> Em cidades como Betim (MG), a implementação do Orçamento Participativo permitiu que a população decidisse como investir os recursos públicos, priorizando projetos que beneficiaram diretamente a comunidade. Outros municípios utilizam plataformas online para consultas públicas, enquetes e recebimento de sugestões, fortalecendo a participação cidadã e a transparência na gestão pública.
> 
> **Capacitação e Microcrédito: Ferramentas para o Sucesso:**
> 
> Acreditamos que o conhecimento é a chave para o crescimento. Por isso, vamos mapear e divulgar cursos de capacitação e oportunidades de microcrédito disponíveis na região.
> 
> *   **Cursos de Empreendedorismo:** Aprenda a gerenciar seu negócio, desde o planejamento até a divulgação.
> *   **Oficinas de Artesanato:** Aprimore suas habilidades e crie produtos ainda mais incríveis.
> *   **Microcrédito:** Encontre linhas de crédito com juros acessíveis para investir no seu negócio.
> 
> **Ações Práticas para Começar Já:**
> 
> 1.  **Cadastre-se na Plataforma:** Crie seu perfil e divulgue seus produtos/serviços.
> 2.  **Participe das Enquetes:** Dê sua opinião sobre as prioridades para o desenvolvimento da cidade.
> 3.  **Sugira Melhorias:** Use a plataforma para reportar problemas e propor soluções.
> 4.  **Voluntarie-se:** Ajude a organizar eventos e projetos que beneficiem a comunidade.
> 
> **Juntos, podemos transformar Três Corações em um exemplo de desenvolvimento econômico local!**


-----------------------------------------------------------

--- Resultado do Agente 4 (Descomplicador de Processos Municipais) ---



> ## Guia Simplificado de Serviços Municipais em Três Corações
> 
> Olá, cidadão tricordiano! Para facilitar seu dia a dia, preparamos um guia prático de como acessar alguns serviços importantes da nossa cidade através da Plataforma Cidadã Integrada.
> 
> **1. Licenças Simplificadas (Ambulante, Pequenas Reformas):**
> 
> Se você deseja trabalhar como ambulante ou precisa realizar pequenas reformas em sua casa ou comércio, siga este passo a passo simplificado:
> 
> *   **Passo 1: Cadastro na Plataforma Cidadã:** Acesse a plataforma (se já não tiver), e faça seu cadastro. Tenha em mãos seus documentos pessoais (RG, CPF, comprovante de residência) e os dados da sua atividade ou do imóvel a ser reformado.
> *   **Passo 2: Solicitação Online:** Na plataforma, procure pela seção de "Licenças e Alvarás". Lá, você encontrará as opções de "Licença para Ambulante" ou "Licença para Pequenas Reformas". Selecione a opção desejada.
> *   **Passo 3: Preenchimento do Formulário:** Preencha o formulário online com as informações solicitadas. Seja o mais claro e detalhado possível na descrição da sua atividade ou da reforma.
> *   **Passo 4: Anexar Documentos:** Digitalize e anexe os documentos necessários. Para ambulantes, geralmente são exigidos RG, CPF, comprovante de residência e comprovante de inscrição no Cadastro Municipal de Ambulantes (se houver). Para pequenas reformas, pode ser necessário apresentar a planta do imóvel e um breve descritivo da obra.
> *   **Passo 5: Pagamento da Taxa (se houver):** A plataforma irá gerar um boleto para o pagamento da taxa de emissão da licença, caso seja aplicável. Efetue o pagamento.
> *   **Passo 6: Acompanhamento da Solicitação:** Acompanhe o andamento da sua solicitação na plataforma. Você receberá notificações sobre o status do processo.
> *   **Passo 7: Recebimento da Licença:** Após a aprovação, a licença será disponibilizada na plataforma para download e impressão. Em alguns casos, poderá ser enviada para o seu endereço.
> 
> **2. Inscrição em Programas Sociais Municipais:**
> 
> Para se inscrever nos programas sociais oferecidos pela Prefeitura de Três Corações, siga estas orientações:
> 
> *   **Passo 1: Informações sobre os Programas:** Acesse a Plataforma Cidadã e procure pela seção de "Programas Sociais". Lá, você encontrará a lista completa dos programas disponíveis, com informações sobre os critérios de elegibilidade, os benefícios oferecidos e os documentos necessários.
> *   **Passo 2: Cadastro no Cadastro Único (CadÚnico):** Para a maioria dos programas sociais, é necessário estar cadastrado no CadÚnico. Se você ainda não possui cadastro, procure o CRAS (Centro de Referência de Assistência Social) mais próximo da sua casa para realizar o cadastro. Leve seus documentos pessoais (RG, CPF, Carteira de Trabalho, comprovante de residência, certidão de nascimento ou casamento) e os documentos de todos os membros da sua família.
> *   **Passo 3: Inscrição Online (se disponível):** Alguns programas sociais permitem a inscrição online através da Plataforma Cidadã. Verifique se o programa desejado oferece essa opção.
> *   **Passo 4: Comparecimento ao CRAS (se necessário):** Em alguns casos, mesmo com a inscrição online, pode ser necessário comparecer ao CRAS para apresentar documentos ou participar de entrevistas. A plataforma informará se essa etapa é necessária.
> *   **Passo 5: Acompanhamento da Inscrição:** Acompanhe o andamento da sua inscrição na plataforma ou diretamente no CRAS.
> 
> **3. Acesso a Outros Serviços Municipais:**
> 
> A Plataforma Cidadã Integrada oferece acesso a diversos outros serviços municipais. Para acessá-los, siga estas orientações gerais:
> 
> *   **Documentação Necessária:** Consulte a lista de documentos necessários para cada serviço na plataforma. Geralmente, são exigidos documentos pessoais (RG, CPF, comprovante de residência), comprovantes de renda (se aplicável) e outros documentos específicos de acordo com o serviço.
> *   **Agendamento Online:** Alguns serviços exigem agendamento prévio. Verifique a disponibilidade de horários e agende seu atendimento na plataforma.
> *   **Atendimento Presencial:** Para serviços que não estão disponíveis online, dirija-se ao setor responsável na Prefeitura ou em outros órgãos municipais. Leve todos os documentos necessários e siga as orientações dos atendentes.
> 
> **4. Propostas de Melhorias nos Fluxos de Atendimento:**
> 
> Para tornar os serviços municipais ainda mais eficientes, sugerimos as seguintes melhorias:
> 
> *   **Ampliação do Atendimento Online:** Expandir a oferta de serviços online, permitindo que os cidadãos resolvam suas demandas sem precisar sair de casa.
> *   **Centralização das Informações:** Criar um sistema unificado de informações sobre todos os serviços municipais, facilitando o acesso e a compreensão dos cidadãos.
> *   **Capacitação dos Servidores:** Oferecer treinamento contínuo aos servidores para que eles possam atender os cidadãos de forma mais eficiente e cordial.
> *   **Canais de Comunicação:** Implementar canais de comunicação eficientes (telefone, e-mail, chat online) para que os cidadãos possam tirar dúvidas e obter informações sobre os serviços.
> *   **Pesquisa de Satisfação:** Realizar pesquisas de satisfação para identificar os pontos fortes e fracos dos serviços e implementar melhorias contínuas.
> 
> Com a colaboração de todos, podemos construir uma Três Corações cada vez melhor para se viver!


-----------------------------------------------------------
✅ Plataforma Cidadã Integrada para Municípios Pequenos criada com sucesso! ✅
